# Train OCR kz text Detector quick example

For train datasets please download last version of ocr datasets [https://nomeroff.net.ua/datasets/](https://nomeroff.net.ua/datasets/). Unpack archive and rename to **./datasets/ocr** .
For examle
```bash
cd ./datasets/ocr
wget https://nomeroff.net.ua/datasets/autoriaNumberplateOcrKz-2019-04-26.zip
unzip autoriaNumberplateOcrKz-2019-04-26.zip
mv autoriaNumberplateOcrKz-2019-04-26 kz
```
or use your own dataset.

In [14]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import keras
keras.backend.clear_session()

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

DATASET_NAME = "kz-box"
VERSION = "1"
MODE = "gpu"
PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "datasets/ocr/", DATASET_NAME)
RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}-{}.h5'.format(DATASET_NAME, VERSION, MODE))

FROZEN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}-{}.pb'.format(DATASET_NAME, VERSION, MODE))

sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet.Base import OCR, convert_keras_to_freeze_pb

In [15]:
class kz_box(OCR):
    def __init__(self):
        OCR.__init__(self)
        # only for usage model
        # in train generate automaticly
        self.letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "E", "H", "K", "M", "O", "P", "T", "X", "Y"]
        
        self.EPOCHS = 1

In [16]:
ocrTextDetector = kz_box()
model = ocrTextDetector.prepare(PATH_TO_DATASET, aug_count=0)

GET ALPHABET
Max plate length in "val": 8
Max plate length in "train": 8
Max plate length in "test": 9
Letters train  {'6', '3', 'J', 'Z', 'H', 'Q', 'I', '2', 'V', 'B', '7', '5', 'M', 'P', 'A', '9', 'K', 'X', 'L', 'D', 'C', 'T', 'O', '0', '1', 'S', 'F', 'E', 'U', 'R', 'W', 'G', '4', 'N', 'Y', '8'}
Letters val  {'6', '3', 'J', 'Z', 'H', 'Q', 'I', '2', 'V', 'B', '7', '5', 'M', 'P', 'A', '9', 'K', 'X', 'L', 'N', 'D', 'C', 'T', 'O', '0', '1', 'S', 'E', 'U', 'R', 'W', 'G', '4', 'F', 'Y', '8'}
Letters test  {'6', '3', 'J', 'Z', 'H', 'Q', 'I', '2', 'V', 'B', '5', '7', 'M', 'P', 'A', '9', 'X', 'L', 'D', 'C', 'T', 'O', '0', '1', 'S', 'F', 'E', 'U', 'R', 'W', 'G', '4', 'Y', 'N', 'K', '8'}
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C D E F G H I J K L M N O P Q R S T U V W X Y Z

EXPLAIN DATA TRANSFORMATIONS
kz_box
Text generator output (data which will be fed into the neutral network):
1) the_input (image)
2) the_labe

In [17]:
#model = ocrTextDetector.load(RESULT_MODEL_PATH)
#RESULT_MODEL_PATH

In [19]:
model = ocrTextDetector.train(mode=MODE, is_random=1)


START TRAINING
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_kz_box (InputLayer)   (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_kz_box[0][0]           
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
_____________________________________________________________________________________________

In [20]:
ocrTextDetector.test(verbose=True)


RUN TEST

Predicted: 		 200909KLA
True: 			 20609NLA

Predicted: 		 1302AP
True: 			 13502APA

Predicted: 		 3785A
True: 			 3785AI

Predicted: 		 35602HUA
True: 			 35602HJA

Predicted: 		 35304AD
True: 			 35304AP

Predicted: 		 0554BC
True: 			 0654BC

Predicted: 		 06302FOA
True: 			 06302FGA

Predicted: 		 3809BLA
True: 			 35809BLA
acc: 0.9587628865979382


In [21]:
ocrTextDetector.save(RESULT_MODEL_PATH, verbose=True)

SAVED TO /var/www/nomeroff-net/models/anpr_ocr_kz-box_1-gpu.h5


In [22]:
RESULT_MODEL_PATH

'/var/www/nomeroff-net/models/anpr_ocr_kz-box_1-gpu.h5'

In [23]:
#RESULT_MODEL_PATH='/mnt/data/var/www/html2/js/nomeroff-net_2/models/anpr_ocr_kz_3-cpu.h5'

In [24]:
model = ocrTextDetector.load(RESULT_MODEL_PATH)

In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_kz_box (InputLayer)   (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_kz_box[0][0]           
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
__________________________________________________________________________________________________
max2 (MaxP

### Convert keras OCR  .h5 model to .pb graph

In [26]:
import keras
keras.backend.clear_session()
model = ocrTextDetector.load(RESULT_MODEL_PATH)
convert_keras_to_freeze_pb(model, FROZEN_MODEL_PATH)

OUTPUT: softmax_kz_box/truediv
INPUT: the_input_kz_box
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_kz_box (InputLayer)   (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_kz_box[0][0]           
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
______________________________________________________

In [ ]:
# import keras
# PATH = "../models/anpr_ocr_ru_3-cpu.h5"

# model = keras.models.load_model(PATH, compile=False)
# model.summary()

# inp = model.get_layer("the_input")
# out = model.get_layer("softmax")

# # set new names
# inp.name = "the_input_ru"
# out.name = "softmax_ru"

# net_inp = inp.input
# net_out = out.output
# model = keras.Model(input=net_inp, output=net_out)

# model.summary()

# model.save(PATH)
